TOPIC: 

Currency Exchange rate forecasting(USD -> INR)

Forecast the USD/INR exchange rate using historical data with a classical time series modeling pipeline and deploy the trained model using FastAPI or Streamlit

In [8]:
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

In [7]:
pip install yfinance


Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/949.2 kB ? eta -:--:--
     ---------------------------------------- 949.2/949.2 kB 6.8 MB/s  0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 1.6/1.6 MB 10.0 MB/s  0:00:00
  Created wheel for multitasking: filename=multitasking-0.0.12-py3-none-any.whl size=15702 sha256=761a0140280bc74dd0b51a088d90902ed09d9f62306f5cad1915beb74b86352d
  Stored in directory: c:\users\

  DEPRECATION: Building 'multitasking' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'multitasking'. Discussion can be found at https://github.com/pypa/pip/issues/6334

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


1. Data Collection
- using yahoo finance api
- last 10 years of data


In [ ]:
end_date = (datetime.now()-timedelta(days=2))

start_date = end_date - timedelta(days=365*10)

print(f"\nData Range: {start_date.date()} to {end_date.date()}")



Data Range: 2015-10-27 to 2025-10-24


In [14]:
usd_inr = yf.download('INR=X', start=start_date, end=end_date, progress=False)

In [15]:
usd_inr

Price,Close,High,Low,Open,Volume
Ticker,INR=X,INR=X,INR=X,INR=X,INR=X
Date,,,,,
2015-10-27,64.901001,65.060997,64.879997,64.899002,0
2015-10-28,65.067001,65.110001,64.820000,65.072998,0
2015-10-29,64.945999,65.410004,64.945999,64.945999,0
2015-10-30,65.444000,65.445000,65.108002,65.445000,0
2015-11-02,65.415001,65.642998,65.260002,65.415001,0
...,...,...,...,...,...
2025-10-20,88.002296,88.011299,87.499603,88.002296,0
2025-10-21,87.879997,88.083603,87.841904,87.879997,0


In [16]:
usd_inr.columns

MultiIndex([( 'Close', 'INR=X'),
            (  'High', 'INR=X'),
            (   'Low', 'INR=X'),
            (  'Open', 'INR=X'),
            ('Volume', 'INR=X')],
           names=['Price', 'Ticker'])

now will select only the “Close” price column from  multi-indexed Yahoo Finance data and rename it to 'USD_INR'.

This creates a clean, single-index DataFrame that’s easier to use in time series analysis.

In [17]:
df = pd.DataFrame({
    'USD_INR': usd_inr[('Close', 'INR=X')]
})


In [18]:
df


,USD_INR
Date,
2015-10-27,64.901001
2015-10-28,65.067001
2015-10-29,64.945999
2015-10-30,65.444000
2015-11-02,65.415001
...,...
2025-10-20,88.002296
2025-10-21,87.879997
2025-10-22,88.001602


In [19]:
print(f"Collected {len(df)} days of USD/INR data")
print(f"Range: {df['USD_INR'].min():.4f} to {df['USD_INR'].max():.4f}")
    

Collected 2603 days of USD/INR data
Range: 63.2650 to 88.8654


the above specifies that data was collected for 2603 market days and not calender days, with minimum value being 64.265 and max being 88.86

In [20]:
#daily data
# Save daily data
df.to_csv('usd_inr_daily_raw.csv')
print(f" Daily data saved: usd_inr_daily_raw.csv ({len(df)} rows)")


 Daily data saved: usd_inr_daily_raw.csv (2603 rows)


In [21]:
print("\nDAILY DATA:")
print(f"  Date Range: {df.index[0].date()} to {df.index[-1].date()}")
print(f"  Total Days: {len(df)}")
print(f"  Trading Days: {df['USD_INR'].notna().sum()}")



DAILY DATA:
  Date Range: 2015-10-27 to 2025-10-24
  Total Days: 2603
  Trading Days: 2603


In [22]:
print("\n Daily Statss:")
print(df['USD_INR'].describe())


 Daily Statss:
count    2603.000000
mean       74.709749
std         7.201498
min        63.264999
25%        68.129749
50%        73.824097
75%        82.418648
max        88.865402
Name: USD_INR, dtype: float64
